# Semantle game

## Word2Vec

In [17]:
from gensim.models import KeyedVectors
import numpy as np
import nltk

# Télécharger les ressources NLTK (si vous ne les avez pas déjà téléchargées)
nltk.download('words')

# Importer la liste des mots communs anglais
from nltk.corpus import words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gregoire.brugere_vit\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package words is already up-to-date!


In [18]:
# Charger le modèle Word2Vec
model = KeyedVectors.load("word2vec.model")

vectors = np.load("word2vec.model.vectors.npy")

In [44]:
import nltk
from nltk.corpus import words, wordnet

# Téléchargement des ressources nécessaires si ce n'est pas déjà fait
nltk.download('words')
nltk.download('wordnet')

# Charger la liste des mots anglais
liste_mots_anglais = words.words()

# Filtrer les mots qui sont des noms communs
noms_communs = set()
for mot in liste_mots_anglais:
    synsets = wordnet.synsets(mot)
    for synset in synsets:
        if synset.pos() == 'n':  # Vérifier si le synset est un nom commun
            noms_communs.add(synset.lemmas()[0].name().lower())

#ensemble_mots_anglais = set(mots_anglais)
ensemble_mots_modele = set(model.key_to_index.keys())

lwords = list(ensemble_mots_modele.intersection(noms_communs))

lwords

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gregoire.brugere_vit\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gregoire.brugere_vit\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package wordnet is already up-to-date!


['benjamin', 'handsomeness', 'malto', 'lantern', 'subsidization', 'amphictyony', 'orthopter', 'daucus', 'gametophore', 'levitation']


['benjamin',
 'handsomeness',
 'lantern',
 'subsidization',
 'levitation',
 'bookbinding',
 'volleyball',
 'sneeze',
 'angel',
 'visa',
 'toda',
 'given',
 'planting',
 'stance',
 'sex',
 'derangement',
 'eightieth',
 'meconium',
 'snifter',
 'feudal_lord',
 'googolplex',
 'vishnu',
 'bruin',
 'poncho',
 'maxwell',
 'reflation',
 'slumber',
 'cleft_lip',
 'synergy',
 'medlar',
 'calender',
 'python',
 'easement',
 'predicament',
 'po',
 'kremlin',
 'radiocarbon',
 'dub',
 'graph',
 'expunction',
 'kidney',
 'frond',
 'router',
 'smallpox',
 'triple',
 'onomatopoeia',
 'alaska',
 'poppy',
 'sense',
 'creditor',
 'californian',
 'celestial_equator',
 'intellectualization',
 'scrabble',
 'semimonthly',
 'travelogue',
 'bowsprit',
 'marl',
 'totalitarian',
 'oeuvre',
 'shrimper',
 'leisure',
 'straggler',
 'correction',
 'lipoprotein',
 'guinea',
 'sudan',
 'burlap',
 'transpose',
 'minaret',
 'venipuncture',
 'schmaltz',
 'sun',
 'translocation',
 'tumescence',
 'confessional',
 'conceit'

## Game

In [34]:
import numpy as np

def distance_cosinus_normalisee(mot1, mot2, modele):
    try:
        vecteur_mot1 = modele.get_vector(mot1)
        vecteur_mot2 = modele.get_vector(mot2)
    except KeyError:
        return "Un ou plusieurs mots ne sont pas présents dans le modèle Word2Vec."
    
    # Calcul du produit scalaire entre les vecteurs
    produit_scalaire = np.dot(vecteur_mot1, vecteur_mot2)
    
    # Calcul des normes des vecteurs
    norme_mot1 = np.linalg.norm(vecteur_mot1)
    norme_mot2 = np.linalg.norm(vecteur_mot2)
    
    # Calcul de la distance cosinus normalisée
    distance = produit_scalaire / (norme_mot1 * norme_mot2)
    
    return distance

# Exemple d'utilisation
# Supposons que "modele" est votre modèle Word2Vec chargé précédemment

mot1 = "cat"
mot2 = "animal"
print("Distance cosinus normalisée entre", mot1, "et", mot2, ":", distance_cosinus_normalisee(mot1, mot2, model))


Distance cosinus normalisée entre cat et animal : 0.59076893


In [54]:
def mots_plus_proches(mot, modele, lwords, nombre=10):
    try:
        mots_proches = modele.most_similar(mot, topn=nombre*2)
        mots_proches_tries = sorted(mots_proches, key=lambda x: x[1], reverse=True)
        mots_proches_filtres = []
        for mot, distance in mots_proches_tries:
            if mot in lwords:
                mots_proches_filtres.append(mot)
                if len(mots_proches_filtres) == nombre:
                    break
        return mots_proches_filtres
    except KeyError:
        print("Le mot spécifié n'est pas présent dans le modèle Word2Vec.")
        return []

    
mots_plus_proches('attainment', model, lwords)

['aspiration',
 'obtainment',
 'advancement',
 'achievability',
 'rurality',
 'proficiency',
 'education',
 'desideratum',
 'implementation',
 'emancipation']

In [57]:
import random

def jouer_semantle(liste_mots, modele, show_answer = False):
    mot_choisi = random.choice(liste_mots)
    if show_answer:
        print("The word to guess is:", mot_choisi)
    lmots_plus_proches = mots_plus_proches(mot_choisi, modele, lwords, 100)
    
    while True:
        guess = input("Guess the word : ")
        guess = guess.replace(" ", "_")
        
        if guess == mot_choisi:
            print("Congratulations! You guessed the word correctly.")
            break
        else:
            distance = distance_cosinus_normalisee(mot_choisi, guess, modele)
            if distance is None:
                print("The entered word is not present in the Word2Vec model.")
            else:
                index = None
                for i, mot in enumerate(lmots_plus_proches):
                    if mot == guess:
                        index = i
                        break
                if index is not None:
                    print(guess + " : " + str(distance) + ", rank : " + str(index))
                else:
                    print(guess + " : " + str(distance))

In [58]:
jouer_semantle(lwords, model, show_answer = True)

The word to guess is: lumbering
Guess the word : timber
timber : 0.15670085
Guess the word : forest
forest : 0.15693888
Guess the word : lumber
lumber : 0.16709332
Guess the word : wood
wood : 0.10388643
Guess the word : tree
tree : 0.09620945
Guess the word : lumbering
Congratulations! You guessed the word correctly.
